<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Gestión en Logística y Cadena de Suministro</strong></h1>
  <h3><strong>Ejercicio 5</strong></h3>
</div>

El sudoku es un juego japonés. El objetivo del juego es rellenar con los números naturales del 1 al 9 cada una de las celdas vacías de una cuadrícula de $9 \times 9$, dividida en 9 sub-cuadrículas de $3 \times 3$, y que ha sido parcialmente rellenada con algunos valores en algunas de las celdas. El objetivo del juego es rellenar las celdas vacías de manera tal que en cada una de las filas, en cada una de las columnas y en cada una de las submatrices no debe haber números repetidos. Se requiere formular el problema de resolver un sudoku como un problema de programación matemática.

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjunto de índices**


- **Filas:** $i \in I = \{1,2,\dots,9\}$
- **Columnas:** $j \in J = \{1,2,\dots,9\}$
- **Dígitos:** $k \in K = \{1,2,\dots,9\}$

Para representar los bloques o subcuadrículas (3x3), se puede definir:

- **Bloques:** $b \in B = \{1,2,\dots,9\}$

  Cada bloque $b$ está compuesto por un conjunto de celdas. Una manera común de indexar los bloques es asignarles de la siguiente forma:

  - Bloque $b$ contiene las celdas:
    $$
    \{ (i,j) \mid i \in I_b,\, j \in J_b \},
    $$
    donde los conjuntos $I_b$ y $J_b$ se definen de acuerdo con la posición del bloque. Por ejemplo:
    - Bloque 1: $I_1 = \{1,2,3\}$, $J_1 = \{1,2,3\}$.
    - Bloque 2: $I_2 = \{1,2,3\}$, $J_2 = \{4,5,6\}$.
    - y así sucesivamente.

También se pueden definir las siguientes funciones para identificar el bloque al que pertenece una celda:
$$
\text{bloque}(i,j) = 3\left(\left\lceil \frac{i}{3} \right\rceil - 1\right) + \left\lceil \frac{j}{3} \right\rceil.
$$

### **Parámetros**

- **Datos iniciales (pistas):**  
  Sea $a_{ij}$ el dígito preasignado en la celda $(i,j)$ si es que existe, y 0 (o un valor nulo) si la celda está vacía.  
  Por ejemplo, en un Sudoku dado, podríamos tener:
  $$
  a_{ij} =
  \begin{cases}
  k, & \text{si la celda } (i,j) \text{ ya contiene el dígito } k, \\
  0, & \text{si la celda } (i,j) \text{ está vacía.}
  \end{cases}
  $$

In [3]:
# definimos el sudoku a resolver:
puzzle = [
    [0, 0, 0,   0, 0, 0,    0, 0, 0],
    [6, 2, 0,   0, 8, 5,    3, 0, 0],
    [8, 0, 7,   0, 2, 4,    0, 1, 0],

    [0, 0, 5,   7, 3, 2,    0, 6, 4],
    [0, 8, 0,   0, 0, 0,    2, 0, 0],
    [0, 0, 0,   0, 0, 0,    0, 0, 0],
    
    [7, 3, 0,   0, 0, 0,    0, 8, 6],
    [0, 0, 0,   0, 0, 0,    0, 0, 0],
    [0, 0, 0,   0, 6, 8,    0, 0, 0]
]

# Índices: filas, columnas y dígitos (0 a 8, para representar 1 a 9)
n = 9
N = range(n) # no necesitamos todas las filas, columnas y dígitos, pero es más fácil así

### **Variables de decisión**

Se define la variable binaria:

$$
x_{ijk} =
\begin{cases}
1, & \text{si se asigna el dígito } k \text{ a la celda } (i,j), \\
0, & \text{en caso contrario.}
\end{cases}
$$

Esta variable toma valor 1 únicamente si en la celda $(i,j)$ se coloca el dígito $k$.

In [4]:
x = {}
for i in N:
    for j in N:
        for k in N:
            x[i, j, k] = solver.BoolVar(f'x_{i}_{j}_{k}') 

### **Función objetivo**

El objetivo es encontrar una solución factible que cumpla con todas las restricciones del Sudoku, por lo que se puede definir una función objetivo trivial, por ejemplo:

$$
\min \; 0.
$$

Por lo mismo, cualquier constante sirve como función objetivo :)

In [5]:
solver.Minimize(0)

### **Restricciones**

Cada celda $(i,j)$ debe contener exactamente un dígito:

$$
\sum_{k \in K} x_{ijk} = 1, \quad \forall \, i \in I, \; \forall \, j \in J.
$$

In [6]:
for i in N:
    for j in N:
        solver.Add(sum(x[i, j, k] for k in N) == 1)

Cada dígito $k$ debe aparecer exactamente una vez en cada fila $i$:

$$
\sum_{j \in J} x_{ijk} = 1, \quad \forall \, i \in I, \; \forall \, k \in K.
$$

In [7]:
for i in N:
    for k in N:
        solver.Add(sum(x[i, j, k] for j in N) == 1)

Cada dígito $k$ debe aparecer exactamente una vez en cada columna $j$:

$$
\sum_{i \in I} x_{ijk} = 1, \quad \forall \, j \in J, \; \forall \, k \in K.
$$


In [8]:
for j in N:
    for k in N:
        solver.Add(sum(x[i, j, k] for i in N) == 1)

Cada dígito $k$ debe aparecer exactamente una vez en cada bloque $b$. Sea $C(b)$ el conjunto de celdas $(i,j)$ que pertenecen al bloque $b$. Entonces:

$$
\sum_{(i,j) \in C(b)} x_{ijk} = 1, \quad \forall \, b \in B, \; \forall \, k \in K.
$$

In [9]:
B = range(3)

for block_i in B:
    for block_j in B:
        for k in N:
            # Añade una restricción al solver indicando que la suma de x[i, j, k] 
            # en el bloque de 3x3 debe ser igual a 1, para que el número k 
            # aparezca una vez en cada bloque de 3x3
            solver.Add(
                sum(
                    x[i, j, k]
                    for i in range(block_i * 3, block_i * 3 + 3)
                    for j in range(block_j * 3, block_j * 3 + 3)
                ) == 1
            )

**Restricciones de Pistas (Datos Iniciales)**

Para las celdas que ya tienen un dígito preasignado, se fija la variable correspondiente:

$$
x_{ij,k} = 1 \quad \text{si } a_{ij} = k, \quad \forall \, (i,j) \text{ tal que } a_{ij} \neq 0.
$$

O de forma equivalente, se puede eliminar la libertad en la celda fijándola a ese valor.

In [10]:
for i in N:
    for j in N:
        # Si el valor en la posición (i, j) del puzzle no es 0
        if puzzle[i][j] != 0:
            # Calcula el índice k restando 1 al valor en la posición (i, j), ya que los índices van de 0 a 8
            k = puzzle[i][j] - 1
            # Añade una restricción al solver indicando que x[i, j, k] debe ser igual a 1
            solver.Add(x[i, j, k] == 1)

### **Resolver**

In [11]:
solver.Solve()

solution = [[0 for _ in N] for _ in N]
for i in N:
    for j in N:
        for k in N:
            if x[i, j, k].solution_value() == 1: #si la variable x[i, j, k] es 1, entonces el dígito k+1 está en la posición (i, j)
                solution[i][j] = k + 1  

for row in solution:
    print(row) #para ver nuestro sudoku resuelto

[3, 1, 4, 6, 9, 7, 5, 2, 8]
[6, 2, 9, 1, 8, 5, 3, 4, 7]
[8, 5, 7, 3, 2, 4, 6, 1, 9]
[1, 9, 5, 7, 3, 2, 8, 6, 4]
[4, 8, 6, 5, 1, 9, 2, 7, 3]
[2, 7, 3, 8, 4, 6, 1, 9, 5]
[7, 3, 2, 4, 5, 1, 9, 8, 6]
[9, 6, 8, 2, 7, 3, 4, 5, 1]
[5, 4, 1, 9, 6, 8, 7, 3, 2]
